Nama : Dion Prayoga

# Proyek Analisis Sentimen Disney+ Hotstar

**Submission Pertama** dari course **Belajar Pengembangan Machine Learning (Dicoding)**.  

## Deskripsi Proyek
Proyek ini berfokus pada **analisis sentimen** terhadap ulasan pengguna aplikasi **Disney+ Hotstar** yang diperoleh dari **Google Play Store**. Proses analisis mencakup:

- **Web Scraping** untuk mengambil data ulasan.
- **Pra-pemrosesan Data** untuk mempersiapkan data untuk analisis.
- **Klasifikasi Sentimen** menggunakan teknik **Machine Learning**.

## Pelatihan Model
Tiga model digunakan dalam proyek ini:
1. **SVM dengan TF-IDF**: Menggunakan model Support Vector Machine (SVM) dengan representasi fitur berbasis TF-IDF untuk memproses teks.
2. **Random Forest dengan TF-IDF**: Model Random Forest yang juga menggunakan representasi fitur berbasis TF-IDF.
3. **Deep Learning dengan LSTM**: Model jaringan saraf berbasis LSTM untuk memproses teks.

## Hasil
Setelah melatih ketiga model, hasil evaluasi menunjukkan bahwa model terbaik dipilih berdasarkan akurasi tertinggi. Perbandingan antara label aktual dan prediksi untuk model dengan kinerja terbaik ditampilkan untuk menunjukkan bagaimana model memprediksi sentimen dalam ulasan.

📚 Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
import re
import string

📥 Membaca Dataset

In [3]:
df = pd.read_csv('reviews_disneyhotstar.csv')

print("Jumlah baris dan kolom:")
print(df.shape)

Jumlah baris dan kolom:
(20000, 11)


🧾 Memuat dan Mengecek Informasi Dataset

In [4]:
df = pd.read_csv('reviews_disneyhotstar.csv')

print("\nInformasi kolom:")
print(df.info())


Informasi kolom:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              20000 non-null  object
 1   userName              20000 non-null  object
 2   userImage             20000 non-null  object
 3   content               20000 non-null  object
 4   score                 20000 non-null  int64 
 5   thumbsUpCount         20000 non-null  int64 
 6   reviewCreatedVersion  13697 non-null  object
 7   at                    20000 non-null  object
 8   replyContent          4729 non-null   object
 9   repliedAt             4729 non-null   object
 10  appVersion            13697 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.7+ MB
None


👀 Menampilkan 5 Baris Pertama Dataset

In [5]:
df = pd.read_csv('reviews_disneyhotstar.csv')

print("\n5 baris pertama dataset:")
print(df.head())


5 baris pertama dataset:
                               reviewId         userName  \
0  395f4a25-241d-411c-9c3b-51a22499ea41  Pengguna Google   
1  e3ae832a-6ffe-4e66-b081-b37e364ce2a4  Pengguna Google   
2  d99927ec-1f07-4af8-b4da-79880797472e  Pengguna Google   
3  6b3eca7c-f203-4c87-89d6-12244e0b7646  Pengguna Google   
4  66cfef11-fdbf-4305-b7b5-ba80214c175a  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0           di hp ada suaranya, di tv gaada suaranya      2              0   
1  lebih baik gratis dari pada susah berlangganan 😑😑      1              0   
2          Jelek bgt nyesel dowlod jangan 

🧹 Pembersihan Data: Menghapus Nilai Null pada Kolom 'userName'

In [6]:
df_cleaned = df.dropna(subset=['userName'])
print(df_cleaned.isnull().sum())


reviewId                    0
userName                    0
userImage                   0
content                     0
score                       0
thumbsUpCount               0
reviewCreatedVersion     6303
at                          0
replyContent            15271
repliedAt               15271
appVersion               6303
dtype: int64


🧼 Pembersihan Teks: Membersihkan Konten Ulasan

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['cleaned_content'] = df['content'].apply(clean_text)

🧠 Klasifikasi Sentimen: Menentukan Sentimen Ulasan

In [8]:
def get_sentiment(text):
    from textblob import TextBlob
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return 'positif'
    elif polarity < 0:
        return 'negatif'
    else:
        return 'netral'

df['label'] = df['cleaned_content'].apply(get_sentiment)

🔀 Membagi Data: Pembagian Data Latih dan Uji

In [9]:
y = LabelEncoder().fit_transform(df['label'])
X = df['cleaned_content']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

🧠 Pemrosesan Fitur dan Penanganan Kelas Tidak Seimbang

In [10]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train_raw)
X_test_tfidf = vectorizer.transform(X_test_raw)

smote = SMOTE(random_state=42)
X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

print("Distribusi label setelah SMOTE:", Counter(y_train_smote))

Distribusi label setelah SMOTE: Counter({np.int64(1): 13957, np.int64(2): 13957, np.int64(0): 13957})


⚙️ Pelatihan dan Evaluasi Model SVM

In [11]:
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_tfidf_smote, y_train_smote)
y_pred_svm = svm.predict(X_test_tfidf)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("\nEvaluasi Model SVM:")
print(f"Accuracy: {accuracy_svm}")
print(classification_report(y_test, y_pred_svm, target_names=['negatif', 'netral', 'positif']))


Evaluasi Model SVM:
Accuracy: 0.981
              precision    recall  f1-score   support

     negatif       0.86      0.80      0.83       128
      netral       0.99      1.00      0.99      3489
     positif       0.97      0.91      0.94       383

    accuracy                           0.98      4000
   macro avg       0.94      0.90      0.92      4000
weighted avg       0.98      0.98      0.98      4000



Model Support Vector Machine (SVM) menunjukkan kinerja yang sangat baik dengan akurasi mencapai 98.1%, yang berarti model dapat mengklasifikasikan hampir 98 dari 100 ulasan dengan benar. Untuk kategori "netral", model memiliki precision dan recall hampir sempurna (0.99 dan 1.00), sementara untuk kategori "positif" juga sangat baik dengan F1-Score 0.94. Namun, untuk kategori "negatif", model sedikit lebih rendah dengan recall 0.80, yang menunjukkan bahwa beberapa ulasan negatif tidak terdeteksi. Meskipun demikian, model secara keseluruhan menunjukkan hasil yang solid dengan precision, recall, dan F1-Score yang tinggi, terutama pada kategori dengan jumlah data yang lebih besar (netral).

⚙️ Pelatihan dan Evaluasi Model Random Forest

In [12]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_tfidf_smote, y_train_smote)
y_pred_rf = rf.predict(X_test_tfidf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("\nEvaluasi Model Random Forest:")
print(f"Accuracy: {accuracy_rf}")
print(classification_report(y_test, y_pred_rf, target_names=['negatif', 'netral', 'positif']))


Evaluasi Model Random Forest:
Accuracy: 0.9525
              precision    recall  f1-score   support

     negatif       0.93      0.58      0.71       128
      netral       0.95      1.00      0.97      3489
     positif       0.95      0.68      0.79       383

    accuracy                           0.95      4000
   macro avg       0.94      0.75      0.83      4000
weighted avg       0.95      0.95      0.95      4000



Model Random Forest memberikan akurasi 95.25%, yang menunjukkan kinerja yang cukup baik dalam mengklasifikasikan ulasan. Precision dan recall untuk kategori "netral" sangat baik dengan precision 0.95 dan recall 1.00, sementara untuk kategori "positif" dan "negatif" hasilnya sedikit lebih rendah. Kategori "negatif" memiliki recall yang rendah (0.58), menunjukkan bahwa banyak ulasan negatif yang tidak terdeteksi dengan baik, meskipun precision-nya tinggi (0.93). Demikian pula, kategori "positif" memiliki recall yang lebih rendah (0.68), namun precision-nya tetap tinggi (0.95). Secara keseluruhan, meskipun ada kesulitan dalam mengklasifikasikan ulasan negatif dan positif, model ini tetap efektif dengan nilai F1-Score yang cukup baik dan akurasi tinggi secara keseluruhan.

🧑‍💻 Tokenisasi dan Padding untuk Data Teks

In [13]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_raw)

X_train_seq = tokenizer.texts_to_sequences(X_train_raw)
X_test_seq = tokenizer.texts_to_sequences(X_test_raw)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

y_train_cat = pd.get_dummies(y_train).values
y_test_cat = pd.get_dummies(y_test).values

Model LSTM

In [14]:
model_lstm = Sequential([
    Embedding(input_dim=max_words, output_dim=100, input_length=max_len),
    LSTM(128, return_sequences=True, kernel_regularizer='l2'),
    Dropout(0.5),
    LSTM(64, kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]

history = model_lstm.fit(
    X_train_pad, y_train_cat,
    epochs=15,
    batch_size=64,
    validation_data=(X_test_pad, y_test_cat),
    callbacks=callbacks
)

oss, accuracy_lstm = model_lstm.evaluate(X_test_pad, y_test_cat)
print("\nEvaluasi Model LSTM:")
print(f"Accuracy: {accuracy_lstm}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 104s 369ms/step - accuracy: 0.8591 - loss: 1.7830 - val_accuracy: 0.8723 - val_loss: 0.4749 - learning_rate: 0.0010
Epoch 2/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 139s 361ms/step - accuracy: 0.8701 - loss: 0.4833 - val_accuracy: 0.8723 - val_loss: 0.4612 - learning_rate: 0.0010
Epoch 3/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 139s 351ms/step - accuracy: 0.8749 - loss: 0.4597 - val_accuracy: 0.8723 - val_loss: 0.4547 - learning_rate: 0.0010
Epoch 4/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 146s 367ms/step - accuracy: 0.8673 - loss: 0.4745 - val_accuracy: 0.8723 - val_loss: 0.4547 - learning_rate: 0.0010
Epoch 5/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 141s 365ms/step - accuracy: 0.8722 - loss: 0.4665 - val_accuracy: 0.8723 - val_loss: 0.4617 - learning_rate: 0.0010
Epoch 6/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 143s 370ms/step - accuracy: 0.8767 - loss: 0.4557 - val_accuracy: 0.8723 - val_loss: 0.4543 - learning_rate: 2.0000e-04
Epoch 7/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 138s 353ms/step - ac

Model LSTM yang telah dilatih menunjukkan akurasi sebesar 87.22% pada data uji. Meskipun akurasi tidak mencapai angka yang sangat tinggi, model ini masih cukup baik dalam mengklasifikasikan ulasan berdasarkan sentimen. Pada proses pelatihan, model menunjukkan penurunan kerugian (loss) yang stabil dari 1.78 di epoch pertama hingga 0.45 pada epoch terakhir, menunjukkan bahwa model mulai lebih akurat dalam memprediksi label sentimen seiring waktu. Selain itu, meskipun ada fluktuasi kecil pada akurasi dan loss antara epoch, model secara keseluruhan menunjukkan kinerja yang konsisten dan baik dalam mengenali pola dalam data ulasan.

## **Kesimpulan Evaluasi Model Sentimen**

Dari evaluasi tiga model yang diterapkan dalam proyek analisis sentimen terhadap ulasan aplikasi Disney+ Hotstar, diperoleh hasil yang cukup menarik:

1. **Model SVM (Support Vector Machine)** memberikan hasil terbaik dengan akurasi **98.1%**. Model ini menunjukkan performa yang sangat baik dalam mengklasifikasikan ulasan dengan presisi tinggi pada kategori **positif** dan **netral**, namun sedikit kurang pada kategori **negatif**, dengan recall yang lebih rendah pada kelas tersebut.

2. **Model Random Forest** menghasilkan akurasi **95.25%**. Meskipun akurasinya lebih rendah dibandingkan dengan SVM, model ini masih menunjukkan kinerja yang baik, dengan presisi yang solid untuk kategori **positif** dan **netral**, namun kurang optimal dalam mengidentifikasi ulasan **negatif** (recall rendah pada kategori ini).

3. **Model LSTM (Long Short-Term Memory)**, meskipun lebih kompleks, menunjukkan akurasi **87.23%**. Model ini memiliki potensi yang lebih baik dalam memproses data teks yang lebih besar dan lebih beragam, namun akurasinya lebih rendah dibandingkan dengan dua model sebelumnya. Model LSTM cenderung lebih efektif dalam menangani urutan kata dan konteks dalam teks, meskipun memerlukan lebih banyak waktu dan sumber daya untuk pelatihan.

Secara keseluruhan, **Model SVM** memberikan hasil terbaik untuk tugas klasifikasi sentimen pada ulasan pengguna Disney+ Hotstar, diikuti oleh **Random Forest** dan **LSTM** yang menunjukkan keunggulan masing-masing dalam hal teknik dan kompleksitas model.

Menyimpan dan Mengunduh Daftar Dependensi Python dalam File `requirements.txt`

In [15]:
!pip freeze > requirements.txt
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>